In [1]:
import pandas as pd
import numpy as np
import pathlib
import os
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
#from keras.api._v2.keras.metrics import F1Score
import tensorflow_addons as tfa
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)
np.set_printoptions(precision=4)
print(tf.version.VERSION)
%load_ext tensorboard

2023-07-27 10:02:54.757752: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-27 10:02:54.760307: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 10:02:54.807986: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 10:02:54.809214: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 10:02:57.472674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

2.13.0


In [6]:
import logging
#logging.basicConfig(level=logging.INFO)
#logger = logging.getLogger(ww.__name__)
#logger.setLevel(logging.INFO)
checkpoint_path = "/home/sharmam8/Tests/experiments/logs/23Jul16_combined/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
my_callbacks = [
tf.keras.callbacks.EarlyStopping(patience=2),
tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
tf.keras.callbacks.TensorBoard(log_dir='./logs'),
tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)
]

In [2]:
import pathlib
all_files =  sorted(str(p) for p in pathlib.Path('/home/sharmam8/Tests/experiments/data').glob("*.csv"))
df = pd.read_csv("/home/sharmam8/Tests/experiments/data/_11_MATRIX.csv")
for i in range(len(all_files)-1):
  temp = pd.read_csv(all_files[i])
  df = pd.concat([df, temp], axis=0)
  #print(len(df.index))
df['ContractionDuration'] = df['Contraction End Time (s)'] - df['Contraction Start Time(s)']
df.head()

,Participant ID,Trial ID,Trial Type,Pulse Width,Contraction Start Time(s),Contraction End Time (s),ContractionNo,Pain Label,Sex,Age,...,Weight,Previous Injury,Fibula Length,Shank Girth,Time(s),EMG(mv),MMG_x,MMG_y,MMG_z,ContractionDuration
0,1,11,1,2.677351,6.506103,9.183454,1,1,1,23,...,50.0,0,35.0,35.0,6.5062,95.524229,1.591994,1.301494,1.546334,2.677351
1,1,11,1,2.677351,6.506103,9.183454,1,1,1,23,...,50.0,0,35.0,35.0,6.5064,95.876228,1.596378,1.301486,1.546180,2.677351
2,1,11,1,2.677351,6.506103,9.183454,1,1,1,23,...,50.0,0,35.0,35.0,6.5066,96.267965,1.600714,1.301478,1.546035,2.677351
3,1,11,1,2.677351,6.506103,9.183454,1,1,1,23,...,50.0,0,35.0,35.0,6.5068,96.670317,1.604998,1.301471,1.545901,2.677351
4,1,11,1,2.677351,6.506103,9.183454,1,1,1,23,...,50.0,0,35.0,35.0,6.5070,97.063615,1.609228,1.301467,1.545781,2.677351


In [3]:
def data_prep(my_state):
  #select features
  X = df[['Trial Type', 'ContractionDuration', 'Pulse Width', \
          'Contraction Start Time(s)', 'Contraction End Time (s)', 'ContractionNo', \
          'Sex', 'Age', 'Height', 'Weight', 'Previous Injury', 'Fibula Length', 'Shank Girth', \
          'Time(s)', 'EMG(mv)', 'MMG_x', 'MMG_y', 'MMG_z', 'ContractionDuration'
          ]]
  y = df['Pain Label']
  #create data split
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=my_state)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=my_state)
  print("Number of samples in the training set: ", len(X_train))
  print("Number of samples in the test set: ", len(X_test))
  print("Number of samples in the validation set: ", len(X_val))
  #scale the data
  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)
  X_val_scaled = scaler.transform(X_val)
  return X_train_scaled, X_test_scaled, X_val_scaled, y_train, y_test, y_val

In [3]:
def create_FCNN_model(my_state, num_feat):
  tf.random.set_seed(my_state)
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(1024, activation='relu', name="relu1Layer"),
      tf.keras.layers.Dense(1024, activation='relu', name="relu2Layer"),
      tf.keras.layers.Dense(1024, activation='relu', name="relu3Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu4Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu5Layer"),
      tf.keras.layers.Dense(1, activation='sigmoid', name="sigmoidLayer")
  ])

  model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.BinaryCrossentropy(name='binary cross entropy'),
        tf.keras.metrics.BinaryIoU(name='binary IoU')
    ]
  )
  return model

def create_CNN_model(my_state, num_feat):
  tf.random.set_seed(my_state)
  model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(100, num_feat, activation='relu', name="convLayer", input_shape=(num_feat, 1)),
      tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding="same"),
      #tf.keras.layers.Conv1D(100, num_feat, activation='relu', name="convLayer2", input_shape=(num_feat, 1)),
      #tf.keras.layers.MaxPooling1D(pool_size=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1024, activation='relu', name="relu1Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu2Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu3Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu4Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu5Layer"),
      tf.keras.layers.Dense(1, activation='sigmoid', name="sigmoidLayer")
  ])

  model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.BinaryCrossentropy(name='binary cross entropy'),
        tf.keras.metrics.BinaryIoU(name='binary IoU')

    ]
  )
  return model

def create_tuned_CNN_model(my_state, factor, rate, num_feat):
  tf.random.set_seed(my_state)
  model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(100, num_feat, activation='relu', name="convLayer", input_shape=(num_feat, 1)),
      tf.keras.layers.MaxPooling1D(pool_size=2, strides=1, padding="same"),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1024, kernel_regularizer=l2(factor), activation='relu', name="relu1Layer"),
      tf.keras.layers.BatchNormalization(synchronized=True),
      Dropout(rate),
      tf.keras.layers.BatchNormalization(synchronized=True),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu2Layer"),
      tf.keras.layers.BatchNormalization(synchronized=True),
      Dropout(rate),
      tf.keras.layers.BatchNormalization(synchronized=True),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu3Layer"),
      tf.keras.layers.BatchNormalization(synchronized=True),
      Dropout(rate),
      tf.keras.layers.BatchNormalization(synchronized=True),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu4Layer"),
      tf.keras.layers.BatchNormalization(synchronized=True),
      Dropout(rate),
      tf.keras.layers.BatchNormalization(synchronized=True),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu5Layer"),
      tf.keras.layers.Dense(1, activation='sigmoid', name="sigmoidLayer")
  ])

  model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.BinaryCrossentropy(name='binary cross entropy'),
        tf.keras.metrics.BinaryIoU(name='binary IoU')

    ]
  )
  return model

def create_LSTM_model(my_state, num_feat):
  tf.random.set_seed(my_state)
  model = tf.keras.Sequential([
      tf.keras.layers.LSTM(256, input_shape=(num_feat, 1)),
      tf.keras.layers.Dense(128, activation='relu', name="relu1Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu2Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu3Layer"),
      tf.keras.layers.Dense(1, activation='sigmoid', name="sigmoidLayer")
  ])
  model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.BinaryCrossentropy(name='binary cross entropy'),
        tf.keras.metrics.BinaryIoU(name='binary IoU')
    ]
  )
  return model

def create_regularized_LSTM_model(my_state, factor, rate, num_feat):
  tf.random.set_seed(my_state)
  model = tf.keras.Sequential([
      tf.keras.layers.LSTM(256, input_shape=(num_feat, 1)),
      #Dropout(rate),
      #tf.keras.layers.LSTM(128, input_shape=(None, 1, 256)),
      Dropout(rate),
      tf.keras.layers.Dense(128, kernel_regularizer=l2(factor), activation='relu', name="relu1Layer"),
      tf.keras.layers.BatchNormalization(synchronized=True),
      Dropout(rate),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu2Layer"),
      tf.keras.layers.BatchNormalization(synchronized=True),
      Dropout(rate),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu3Layer"),
      tf.keras.layers.BatchNormalization(synchronized=True),
      Dropout(rate),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu4Layer"),
      tf.keras.layers.Dense(1, activation='sigmoid', name="sigmoidLayer")
      #tf.keras.layers.Dense(1, activation='relu', name="reluLayer")
  ])

  model.compile(
      loss=tf.keras.losses.binary_crossentropy,
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
      metrics=[
          tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.Precision(name='precision'),
          tf.keras.metrics.Recall(name='recall'),
          tf.keras.metrics.BinaryCrossentropy(name='binary cross entropy'),
          tf.keras.metrics.BinaryIoU(name='binary IoU')
      ]
  )
  return model

def create_combined_model(my_state, factor, rate, num_feat):
  tf.random.set_seed(my_state)
  model = tf.keras.Sequential([
      ### BUILD FEATURE EXTRACTOR

      tf.keras.layers.Conv1D(100, num_feat, activation='relu', name="convLayer", input_shape=(num_feat, 1)),
      tf.keras.layers.MaxPooling1D(pool_size=2),
      tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'),
      tf.keras.layers.MaxPooling1D(pool_size=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1024, activation='relu', name="relu1Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu2Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu3Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu4Layer"),
      tf.keras.layers.Dense(256, activation='relu', name="relu5Layer"),

      ###FEED INTO LSTM
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024, return_sequences=True)),
      #tf.keras.layers.LSTM(units=128, return_sequences=True),
      Dropout(rate),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)),
      Dropout(rate),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
      Dropout(rate),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
      Dropout(rate),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
      Dropout(rate),
      tf.keras.layers.Dense(128, kernel_regularizer=l2(factor), activation='relu', name="relu1Layer"),
      Dropout(rate),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu2Layer"),
      Dropout(rate),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu3Layer"),
      Dropout(rate),
      tf.keras.layers.Dense(256, kernel_regularizer=l2(factor), activation='relu', name="relu4Layer"),
      tf.keras.layers.Dense(1, activation='sigmoid', name="sigmoidLayer")
  ])

  model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.BinaryCrossentropy(name='binary cross entropy'),
        tf.keras.metrics.BinaryIoU(name='binary IoU')

    ]
  )
  return model


####################### TRAINING
def train_model(model, X_train_scaled, y_train, num_epochs, checkpoint_dir, t_callbacks):
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=checkpoint_dir, histogram_freq=1)
  print("Training the neural network.. ")
  history = model.fit(X_train_scaled, y_train, batch_size=32, epochs=num_epochs, callbacks=t_callbacks)
  model.summary()
  return history

######################## VISUALS
def visualize_performance(num_epochs, history):
  import matplotlib.pyplot as plt
  from matplotlib import rcParams
  rcParams['figure.figsize'] = (18, 8)
  rcParams['axes.spines.top'] = False
  rcParams['axes.spines.right'] = False
  plt.plot(
      np.arange(1, num_epochs+1),
      history.history['loss'], label='Loss'
  )
  plt.plot(
      np.arange(1, num_epochs+1),
      history.history['accuracy'], label='Accuracy'
  )
  plt.plot(
      np.arange(1, num_epochs+1),
      history.history['precision'], label='Precision'
  )
  plt.plot(
      np.arange(1, num_epochs+1),
      history.history['recall'], label='Recall'
  )
  plt.title('Evaluation metrics', size=20)
  plt.xlabel('Epoch', size=14)
  plt.legend()


############### PERFORMANCE
def evaluate_performance(model, X_test_scaled, y_test):
  predictions = model.predict(X_test_scaled)
  prediction_classes = [
      1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
  ]
  prediction_classes = [
      1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
  ]
  from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

  print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.4f}')
  print(f'Precision: {precision_score(y_test, prediction_classes):.4f}')
  print(f'Recall: {recall_score(y_test, prediction_classes):.4f}')
  print(f'F1 Score: {f1_score(y_test, prediction_classes):.4f}')

In [ ]:
#setup for training
checkpoint_path = "/home/sharmam8/Tests/experiments/logs/23Jul27_tuned_LSTM_s10_repeated/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

my_callbacks = [
#tf.keras.callbacks.EarlyStopping(patience=2),
#tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
tf.keras.callbacks.TensorBoard(log_dir='./logs'),
tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, \
                                                save_weights_only=True, \
                                                verbose=1)
]



from sklearn.model_selection import LeaveOneGroupOut
X = df[['Trial Type', 'ContractionDuration', 'Pulse Width', \
        'Contraction Start Time(s)', 'Contraction End Time (s)', 'ContractionNo', \
        'Sex', 'Age', 'Height', 'Weight', 'Previous Injury', 'Fibula Length', 'Shank Girth', \
        'Time(s)', 'EMG(mv)', 'MMG_x', 'MMG_y', 'MMG_z', 'ContractionDuration'
        ]]
y = df['Pain Label']
groups = df['Participant ID']
logo = LeaveOneGroupOut()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

my_state = 999
num_epochs = 5
num_feat = 19
factor = 1e-5
rate = 0.3
#create model
#model = create_FCNN_model(my_state, num_feat)
#model = create_LSTM_model(my_state, num_feat)
#model = create_CNN_model(my_state, num_feat)
model = create_regularized_LSTM_model(my_state, factor, rate, num_feat)
#model = create_tuned_CNN_model(my_state, factor, rate, num_feat)
model.summary()


n = 10
X_train = X[df['Participant ID'] != n]
y_train = y[df['Participant ID'] != n]
X_test = X[df['Participant ID'] == n]
y_test = y[df['Participant ID'] == n]
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
history = train_model(model, X_train_scaled, y_train, num_epochs, checkpoint_dir, my_callbacks)
evaluate_performance(model, X_test_scaled, y_test)

# for n in subjects:
# X_train = X[df['Participant ID'] != n]
# y_train = y[df['Participant ID'] != n]
# X_test = X[df['Participant ID'] == n]
# y_test = y[df['Participant ID'] == n]
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# history = train_model(model, X_train_scaled, y_train, num_epochs, checkpoint_dir, cp_callback)
# evaluate_performance(model, X_test_scaled, y_test)



BREAK

In [ ]:
my_state = 86
num_epochs = 1
num_feat = 19
#create model
X_train_scaled, X_test_scaled, X_val_scaled, y_train, y_test, y_val = data_prep(my_state)
model = create_FCNN_model(my_state, num_feat)
#model = create_combined_model(my_state, 1e-5, 0.3, num_feat)
history = train_model(X_train_scaled, y_train, num_epochs)
evaluate_performance(model, X_test_scaled, y_test)

In [17]:
filename='log.csv'
history_logger=tf.keras.callbacks.CSVLogger(filename, separator=",", append=True)